In [1]:
import openai
from qdrant_client import QdrantClient

In [2]:
def get_embedding(text, model="text-embedding-3-small"):
    response= openai.embeddings.create(
    input="random text",
    model="text-embedding-3-small"
    )
    return response.data[0].embedding

### Retrieval function

In [5]:
qdrant_client =QdrantClient(url="http://localhost:6333")

In [11]:
def retrieve_data(query, qdrant_client,k=5):
    query_embedding=get_embedding(query)
    results=qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )
    retrieved_context_ids=[]
    retrieved_context=[]
    similarity_scores=[]
    retrieved_context_ratings=[]
    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)
    return {
        "retrieved_context_ids":retrieved_context_ids,
        "retrieved_context":retrieved_context,
        "retrieved_context_ratings":retrieved_context_ratings,
        "similarity_scores":similarity_scores
        }

In [12]:
retrieved_context=retrieve_data("What kind of earphones can i get?",qdrant_client, k=10)

In [13]:
retrieved_context

{'retrieved_context_ids': ['B09PRK49JH',
  'B0BF18F6R7',
  'B0BP9Z159S',
  'B0BB1YKXL1',
  'B08BR9X387',
  'B0BNHVLF7G',
  'B09LVX3XW2',
  'B0CGM29ZL7',
  'B0CH6P8DYF',
  'B0C4DBSWGW'],
 'retrieved_context': ['YUNZII AKKO Neon 157 Keycaps ASA Profile Keycap Set Double-Shot PBT Keycap Gift Set for Mechanical Keyboard (157 Keys, ASA NEON) 【Stand Out Aesthetic Color Set 】 Inspired by the vivid colors of Miami sunsets, the Akko Neon keycap color is designed by the palette combines cool tones of neon cyan, turquoise blue, purple, and hot pink. With the simple and balanced keycap summery color scheme, the AKKO Neon keycap set enliven your typing and brings more excitement and energy for you. 【Double Shot 85% PBT Keycaps】The Akko Neon Keycap Set delivers in quality with ultra-durable dye-sublimated PBT plastic whose legends will never wear away. The Double-Shot PBT keycaps enhances the overall durability with water and dust resistance. 【Exquisite Reusable Gift package】 With playful functional

### Formatting retrieved context function

In [15]:
def process_context(context):
    formatted_context=""
    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context+=f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context

In [20]:



preprocessed_context=process_context(retrieved_context)

In [22]:
print(preprocessed_context)

- ID: B09PRK49JH, rating: 4.5, description: YUNZII AKKO Neon 157 Keycaps ASA Profile Keycap Set Double-Shot PBT Keycap Gift Set for Mechanical Keyboard (157 Keys, ASA NEON) 【Stand Out Aesthetic Color Set 】 Inspired by the vivid colors of Miami sunsets, the Akko Neon keycap color is designed by the palette combines cool tones of neon cyan, turquoise blue, purple, and hot pink. With the simple and balanced keycap summery color scheme, the AKKO Neon keycap set enliven your typing and brings more excitement and energy for you. 【Double Shot 85% PBT Keycaps】The Akko Neon Keycap Set delivers in quality with ultra-durable dye-sublimated PBT plastic whose legends will never wear away. The Double-Shot PBT keycaps enhances the overall durability with water and dust resistance. 【Exquisite Reusable Gift package】 With playful functional packaging, the Akko Neon Keycap set is not only an excellent gift in any occasions, also has the significance of collection. You can collect keycaps into the exquisi

### Create Promt function

In [18]:

def build_prompt(preprocessed_context, question):
    prompt=f"""
    You are a shopping assistant that can answer question about the products in stock.
    You will be given a question and a list of context.
    Instructions:
    - You need to answer questions based on the provided context only
    - Never use the word context and rfer to it as the available products.process_context

    Context:
    {preprocess_context}

    Question:
    {question}
    """

    return prompt



In [23]:
prompt=build_prompt(preprocessed_context, "what kind of earphones can I get")

In [24]:
print (prompt)


    You are a shopping assistant that can answer question about the products in stock.
    You will be given a question and a list of context.
    Instructions:
    - You need to answer questions based on the provided context only
    - Never use the word context and rfer to it as the available products.process_context

    Context:
    - ID: B09PRK49JH, rating: 4.5, description: YUNZII AKKO Neon 157 Keycaps ASA Profile Keycap Set Double-Shot PBT Keycap Gift Set for Mechanical Keyboard (157 Keys, ASA NEON) 【Stand Out Aesthetic Color Set 】 Inspired by the vivid colors of Miami sunsets, the Akko Neon keycap color is designed by the palette combines cool tones of neon cyan, turquoise blue, purple, and hot pink. With the simple and balanced keycap summery color scheme, the AKKO Neon keycap set enliven your typing and brings more excitement and energy for you. 【Double Shot 85% PBT Keycaps】The Akko Neon Keycap Set delivers in quality with ultra-durable dye-sublimated PBT plastic whose legen

### Generate answer function

In [25]:
def generate_answer(prompt):
    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role":"system", "content": prompt}],
        reasoning_effort="minimal"
    )
    return response.choices[0].message.content

In [26]:
print(generate_answer(prompt))

From the listed items, there are bone conduction headphones available:

- Siniffo Upgraded Bone Conduction Headphones Wireless Bluetooth 5.3 Open Ear Sports Earphones with Mic (model X15)

They are open-ear bone conduction headphones suitable for workouts and sports, with Bluetooth 5.3, IP56 rating, 8 hours of battery, included accessories, and a 2-year warranty. If you’re looking for regular in-ear or over-ear earphones, those aren’t included in the provided items.


### Combined RAG Pipeline

In [27]:
def rag_pipeline(question, top_k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    return answer

In [28]:
print(rag_pipeline("What kind of earphones can I get with ratings above 4.5?"))

Here are earphones with ratings above 4.5 in the list:

- Siniffo Upgraded Bone Conduction Headphones Wireless Bluetooth 5.3 Open Ear Sports Earphones with Mic
  - Rating: 4.0 (Note: this is not above 4.5, so exclude)

- (No other earphones with rating above 4.5 are listed.)

So, among the items shown, there aren’t any earphones rated above 4.5. If you’re flexible on rating, the Siniffo bone conduction headphones are available at 4.0, and there are other non-earphone items in the list. If you’d like, I can help find earphones that meet your rating preference from the full catalog.
